# Resonator Sprectoscopy

In [ ]:
from qibolab.paths import qibolab_folder
from qibolab.instruments.zhinst import SHFQC_QA

# runcard = "/home/admin/Juan/qibolab/src/qibolab/runcards/zhinst.yml"
runcard = qibolab_folder / "runcards" / "zhinst.yml"
use_emulation = False
Device = SHFQC_QA('EL_ZURO', "DEV12146", runcard, use_emulation=use_emulation)

In [ ]:
from qibocal.calibrations.characterization.utils import variable_resolution_scanrange
from qibocal.data import DataUnits
from qibolab.pulses import Pulse, PulseSequence
import laboneq.simple as lo
import numpy as np


def resonator_spectroscopy(
    Device,
    qubit: int,
    start,
    stop,
    step,
    software_averages,
):
    
    sequence = PulseSequence() 
    ro_pulse = Device.create_qubit_readout_pulse(qubit, start=0)
    # ro_pulse = Device.create_qubit_readout_pulse2(7.82e9 ,qubit, start=0)
    sequence.add(ro_pulse)

    # Base_if = Device.instruments["shfqc_qa"]["settings"]["if_frequency"]
    Base_lo = Device.instruments["shfqc_qa"]["settings"]["lo_frequency"]

    # frequency_range = (
    #     variable_resolution_scanrange(
    #         lowres_width, lowres_step, highres_width, highres_step
    #     )
    # )
    
    frequency_range = np.arange(start, stop, step)
    
    fast_sweep_data = DataUnits(
        name=f"sweep_q{qubit}", quantities={"frequency": "Hz"}
    )
    
    for _ in range(software_averages):
        for freq in frequency_range:

            Device.reload_settings()
            # Device.instruments["shfqc_qa"]["settings"]["lo_frequency"] = freq + Base_lo
            Device.instruments["shfqc_qa"]["settings"]["if_frequency"] = freq
            # Device.native_gates["single_qubit"][qubit]["MZ"]["frequency"] = freq
            Device.apply_settings()
            
            # msr, phase, i, q = Device.execute_pulse_sequence(sequence)
            msr, phase, i, q = Device.execute_pulse_sequence_NoSamples(sequence)
            print(msr, phase, i , q)
            
            results = {
                "MSR[V]": msr,
                "i[V]": i,
                "q[V]": q,
                "phase[rad]": phase,
                "frequency[Hz]": freq + Base_lo,
            }
            
            print(Device.calib[f"/logical_signal_groups/q{0}/measure_line"])
            
            fast_sweep_data.add(results)

    # Plot simulated output signals
    # if use_emulation == True:
    #     Device.plot_output_signals()
    # else:
    #     lo.show_pulse_sheet("Pulse_sheet",Device.exp)
            
    return fast_sweep_data

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def frequency_msr_phase__fast_precision(data):
        
    data_fast = data

    # try:
    #     data_fit = Data.load_data(folder, routine, format, f"fit_q{qubit}")
    # except:
    #     data_fit = Data(
    #         quantities=[
    #             "popt0",
    #             "popt1",
    #             "popt2",
    #             "popt3",
    #             "label1",
    #             "label2",
    #         ]
    #     )

    fig = make_subplots(
        rows=1,
        cols=2,
        horizontal_spacing=0.1,
        vertical_spacing=0.1,
        subplot_titles=(
            "MSR (V)",
            "phase (rad)",
        ),
    )

    fig.add_trace(
        go.Scatter(
            x=data_fast.get_values("frequency", "GHz"),
            y=data_fast.get_values("MSR", "uV"),
            name="Fast",
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=data_fast.get_values("frequency", "GHz"),
            y=data_fast.get_values("phase", "rad"),
            name="Fast",
        ),
        row=1,
        col=2,
    )
    
    # if len(data_fast) > 0 and len(data_fit) > 0:
    #     freqrange = np.linspace(
    #         min(data_fast.get_values("frequency", "GHz")),
    #         max(data_fast.get_values("frequency", "GHz")),
    #         2 * len(data_fast),
    #     )
    #     params = [i for i in list(data_fit.df.keys()) if "popt" not in i]
    #     fig.add_trace(
    #         go.Scatter(
    #             x=freqrange,
    #             y=lorenzian(
    #                 freqrange,
    #                 data_fit.get_values("popt0"),
    #                 data_fit.get_values("popt1"),
    #                 data_fit.get_values("popt2"),
    #                 data_fit.get_values("popt3"),
    #             ),
    #             name="Fit",
    #             line=go.scatter.Line(dash="dot"),
    #         ),
    #         row=1,
    #         col=1,
    #     )
    #     fig.add_annotation(
    #         dict(
    #             font=dict(color="black", size=12),
    #             x=0,
    #             y=-0.25,
    #             showarrow=False,
    #             text=f"The estimated {params[0]} is {data_fit.df[params[0]][0]:.1f} Hz.",
    #             textangle=0,
    #             xanchor="left",
    #             xref="paper",
    #             yref="paper",
    #         )
    #     )
    #     fig.add_annotation(
    #         dict(
    #             font=dict(color="black", size=12),
    #             x=0,
    #             y=-0.30,
    #             showarrow=False,
    #             text=f"The estimated {params[1]} is {data_fit.df[params[1]][0]:.3f} uV.",
    #             textangle=0,
    #             xanchor="left",
    #             xref="paper",
    #             yref="paper",
    #         )
    #     )
    fig.update_layout(
        showlegend=True,
        uirevision="0",  # ``uirevision`` allows zooming while live plotting
        xaxis_title="Frequency (GHz)",
        yaxis_title="MSR (uV)",
        xaxis2_title="Frequency (GHz)",
        yaxis2_title="Phase (rad)",
    )
    return fig

High Power

In [ ]:
data = resonator_spectroscopy(
    Device,
    qubit = 0,
    start = 10e6,
    stop = 30e6,
    step = 2e6,
    software_averages=1,
    )

frequency_msr_phase__fast_precision(data)

Low Power

In [ ]:
data = resonator_spectroscopy(
    Device,
    qubit = 0,
    start = 10e6,
    stop = 40e6,
    step = 1e6,
    software_averages=1,
    )

frequency_msr_phase__fast_precision(data)

Check RS

In [ ]:
resonator_frequency = data.get_values("frequency", "hertz")[np.argmax(data.df["MSR"])]
freq = resonator_frequency -  Device.instruments["shfqc_qa"]["settings"]["lo_frequency"]
freq

In [ ]:
Device.reload_settings()
Device.instruments["shfqc_qa"]["settings"]["if_frequency"] = freq
Device.apply_settings()

In [ ]:
sequence = PulseSequence() 
ro_pulse = Device.create_qubit_readout_pulse(0, start=0)
sequence.add(ro_pulse)
    
msr, phase, i, q = Device.execute_pulse_sequence_NoSamples(sequence)
print(msr, phase, i , q)

With Sweeper (WIP)

In [ ]:
# frequency range of spectroscopy scan - around expected centre frequency as defined in qubit parameters
spec_range = 100_000_000
num_points = 5000
num_averages = 1

qubit = 0
resonator_frequency = Device.characterization["single_qubit"][qubit]["resonator_freq"]
resonator_frequency

In [ ]:
from qibocal.calibrations.characterization import resonator_spectroscopy_Z
from qibocal.data import DataUnits

resonator_spectroscopy_Z.resonator_spectroscopy_Z(Device,
    0,
    spec_range,
    num_points,
    num_averages,
    )

In [ ]:
data=DataUnits.load_data('reports', 'Zurich', "csv", f"Sweep_q0")

frequency_msr_phase__fast_precision(data)

--------------------------------------------------

# Qubit Spectroscopy

In [ ]:
from qibolab.paths import qibolab_folder
from qibolab.instruments.zhinst import SHFQC_QA

# runcard = "/home/admin/Juan/qibolab/src/qibolab/runcards/zhinst.yml"
runcard = qibolab_folder / "runcards" / "zhinst.yml"
use_emulation = False
Device = SHFQC_QA('EL_ZURO', "DEV12146", runcard, use_emulation=use_emulation)

In [ ]:
sequence = PulseSequence() 
ro_pulse = Device.create_qubit_readout_pulse(0, start=0)
sequence.add(ro_pulse)
    
msr, phase, i, q = Device.execute_pulse_sequence_NoSamples(sequence)
print(msr, phase, i , q)

In [ ]:
import numpy as np
from qibolab.pulses import Pulse, PulseSequence, ReadoutPulse, PulseShape
from qibocal.data import DataUnits
import laboneq.simple as lo

def qubit_spectroscopy(
    Device,
    qubit: int,
    start,
    end,
    step,
    software_averages,
):

    # Device.reload_settings()

    sequence = PulseSequence()
    qd_pulse = Device.create_qubit_drive_pulse(qubit, start=0, duration=1000)
    ro_pulse = Device.create_qubit_readout_pulse(qubit, start=1000)
    sequence.add(qd_pulse)
    sequence.add(ro_pulse)

    center_frequency = Device.instruments["shfqc_qc"]["settings"]["lo_frequency"]

    freqrange = np.arange(start, end, step)

    # platform.ro_port[qubit].lo_frequency = (
    #     platform.characterization["single_qubit"][qubit]["resonator_freq"]
    #     - ro_pulse.frequency
    # )

    data = DataUnits(name=f"sweep_q{qubit}", quantities={"frequency": "Hz"})
    
    for _ in range(software_averages):
        for freq in freqrange:
            
            
            Device.reload_settings()
            
            # Device.instruments["shfqc_qa"]["settings"]["if_frequency"] = 0
            # Device.native_gates["single_qubit"][0]["MZ"]["frequency"] = 0
            
            Device.instruments["shfqc_qc"]["settings"]["if_frequency"] = freq
            Device.apply_settings()
            
            # platform.qd_port[qubit].lo_frequency = freq - qd_pulse.frequency
            # msr, phase, i, q = platform.execute_pulse_sequence(sequence)[
            #     ro_pulse.serial
            # ]
            
            # msr, phase, i, q = Device.execute_pulse_sequence(sequence)
            msr, phase, i, q = Device.execute_pulse_sequence_NoSamples(sequence)
            print(msr, phase, i , q)
            
            results = {
                "MSR[V]": msr,
                "i[V]": i,
                "q[V]": q,
                "phase[rad]": phase,
                "frequency[Hz]": freq + center_frequency ,
            }
            
            data.add(results)

    # Plot output signals
    if use_emulation == True:
        Device.plot_output_signals()
    else:
        lo.show_pulse_sheet("Pulse_sheet",Device.exp)

    if Device.resonator_type == "3D":
        qubit_frequency = data.get_values("frequency", "Hz")[
            np.argmin(data.get_values("MSR", "V"))
        ]
    else:
        qubit_frequency = data.get_values("frequency", "Hz")[
            np.argmax(data.get_values("MSR", "V"))
        ]

    return data


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def frequency_msr_phase__fast_precision(data):
        
    data_fast = data

    # try:
    #     data_fit = Data.load_data(folder, routine, format, f"fit_q{qubit}")
    # except:
    #     data_fit = Data(
    #         quantities=[
    #             "popt0",
    #             "popt1",
    #             "popt2",
    #             "popt3",
    #             "label1",
    #             "label2",
    #         ]
    #     )

    fig = make_subplots(
        rows=1,
        cols=2,
        horizontal_spacing=0.1,
        vertical_spacing=0.1,
        subplot_titles=(
            "MSR (V)",
            "phase (rad)",
        ),
    )

    fig.add_trace(
        go.Scatter(
            x=data_fast.get_values("frequency", "GHz"),
            y=data_fast.get_values("MSR", "uV"),
            name="Fast",
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=data_fast.get_values("frequency", "GHz"),
            y=data_fast.get_values("phase", "rad"),
            name="Fast",
        ),
        row=1,
        col=2,
    )
    
    # if len(data_fast) > 0 and len(data_fit) > 0:
    #     freqrange = np.linspace(
    #         min(data_fast.get_values("frequency", "GHz")),
    #         max(data_fast.get_values("frequency", "GHz")),
    #         2 * len(data_fast),
    #     )
    #     params = [i for i in list(data_fit.df.keys()) if "popt" not in i]
    #     fig.add_trace(
    #         go.Scatter(
    #             x=freqrange,
    #             y=lorenzian(
    #                 freqrange,
    #                 data_fit.get_values("popt0"),
    #                 data_fit.get_values("popt1"),
    #                 data_fit.get_values("popt2"),
    #                 data_fit.get_values("popt3"),
    #             ),
    #             name="Fit",
    #             line=go.scatter.Line(dash="dot"),
    #         ),
    #         row=1,
    #         col=1,
    #     )
    #     fig.add_annotation(
    #         dict(
    #             font=dict(color="black", size=12),
    #             x=0,
    #             y=-0.25,
    #             showarrow=False,
    #             text=f"The estimated {params[0]} is {data_fit.df[params[0]][0]:.1f} Hz.",
    #             textangle=0,
    #             xanchor="left",
    #             xref="paper",
    #             yref="paper",
    #         )
    #     )
    #     fig.add_annotation(
    #         dict(
    #             font=dict(color="black", size=12),
    #             x=0,
    #             y=-0.30,
    #             showarrow=False,
    #             text=f"The estimated {params[1]} is {data_fit.df[params[1]][0]:.3f} uV.",
    #             textangle=0,
    #             xanchor="left",
    #             xref="paper",
    #             yref="paper",
    #         )
    #     )
    fig.update_layout(
        showlegend=True,
        uirevision="0",  # ``uirevision`` allows zooming while live plotting
        xaxis_title="Frequency (GHz)",
        yaxis_title="MSR (uV)",
        xaxis2_title="Frequency (GHz)",
        yaxis2_title="Phase (rad)",
    )
    return fig

In [ ]:
# runcard = "/home/admin/Juan/qibolab/src/qibolab/runcards/zhinst.yml"
runcard = qibolab_folder / "runcards" / "zhinst.yml"

use_emulation = False
data = qubit_spectroscopy(
    Device = SHFQC_QA('EL_ZURO', "DEV12146", runcard, use_emulation=use_emulation),
    qubit = 0,
    start = -30_000_000,
    end = -10_000_000,
    step = 500_000,
    software_averages=1,
    )

frequency_msr_phase__fast_precision(data)

In [ ]:
qubit_frequency = data.get_values("frequency", "hertz")[np.argmin(data.df["MSR"])]
freq = qubit_frequency -  Device.instruments["shfqc_qc"]["settings"]["lo_frequency"]
freq

Check QS

In [ ]:
Device.reload_settings()
Device.instruments["shfqc_qc"]["settings"]["if_frequency"] = freq
Device.apply_settings()

In [ ]:
Device.calib[f"/logical_signal_groups/q{0}/drive_line"]

In [ ]:
sequence = PulseSequence()
qd_pulse = Device.create_qubit_drive_pulse(0, start=0, duration=1000) 
ro_pulse = Device.create_qubit_readout_pulse(0, start=1000)
sequence.add(qd_pulse)
sequence.add(ro_pulse)
    
msr, phase, i, q = Device.execute_pulse_sequence_NoSamples(sequence)
print(msr, phase, i , q)

With Swepeer (Fix Sweeper code parameters: WIP)

In [ ]:
# frequency range of spectroscopy scan - around expected centre frequency as defined in qubit parameters
spec_range = 20_000_000
num_points = 1000
num_averages = 1

qubit = 0

In [ ]:
from qibocal.calibrations.characterization import qubit_spectroscopy_Z
from qibocal.data import DataUnits

qubit_spectroscopy_Z.qubit_spectroscopy_Z(Device,
    0,
    spec_range,
    num_points,
    num_averages,
    )

In [ ]:
data=DataUnits.load_data('reports', 'ZurichQS', "csv", f"Sweep_q0")
frequency_msr_phase__fast_precision(data)